In [1]:
import dask.array as da
import xarray as xr
import numpy as np
from metpy.units import units

/home/mitch/mamba/envs/pygeo/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


## I/O

In [2]:
zarr_path = "~/repos/ucla/classes/2025_winter/geog256/proj1/data/era5_los_angeles.zarr"
ds = xr.open_dataset(zarr_path, engine="zarr", chunks="auto")
ds

<xarray.Dataset> Size: 10GB
Dimensions:     (valid_time: 648988, latitude: 26, longitude: 30)
Coordinates:
  * latitude    (latitude) float64 208B 38.78 38.53 38.28 ... 33.03 32.78 32.53
  * longitude   (longitude) float64 240B -124.4 -124.2 -123.9 ... -117.4 -117.2
  * valid_time  (valid_time) datetime64[ns] 5MB 1951-01-01 ... 2025-01-13T03:...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    d2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-16T21:06 GRIB to CDM+CF via cfgrib-0.9.1...

## Add derived variables

In [5]:
def calculate_rh(t, td):
    return 100 * da.exp((17.625 * (td - t)) / (243.04 + (td - t)))

ds["rh"] = calculate_rh(ds["t2m"], ds["d2m"])
ds["wind"] = da.sqrt(ds["u10"] ** 2 + ds["v10"] ** 2)
ds

<xarray.Dataset> Size: 14GB
Dimensions:     (valid_time: 648988, latitude: 26, longitude: 30)
Coordinates:
  * latitude    (latitude) float64 208B 38.78 38.53 38.28 ... 33.03 32.78 32.53
  * longitude   (longitude) float64 240B -124.4 -124.2 -123.9 ... -117.4 -117.2
  * valid_time  (valid_time) datetime64[ns] 5MB 1951-01-01 ... 2025-01-13T03:...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    d2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    rh          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    wind        (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-16T21:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
def calc_fosberg(temp, rh, wind):
    # Convert temperature from degrees C to degrees F
    temp = temp * 9 / 5 + 32

    # Convert wind speed from m/s to miles per hour
    wind = wind * 2.23694

    # Initialize m with NaNs
    rh = da.asarray(rh)
    m = da.full_like(rh, np.nan, dtype=rh.dtype)


    # Compute m based on humidity ranges
    f1 = rh <= 10
    f2 = (rh > 10) & (rh <= 50)
    f3 = rh > 50

    m = da.where(f1, 0.03229 + 0.281073 * rh - 0.000578 * rh * temp, m)
    m = da.where(f2, 2.22749 + 0.160107 * rh - 0.01478 * temp, m)
    m = da.where(f3, 21.0606 + 0.005565 * rh ** 2 - 0.00035 * rh * temp - 0.483199 * rh, m)

    # Compute n
    n = 1 - 2 * (m / 30) + 1.5 * (m / 30) ** 2 - 0.5 * (m / 30) ** 3

    # Compute Fosberg Fire Weather Index
    fwi = n * ((1 + wind ** 2) ** 0.5) / 0.3002

    return fwi

In [7]:
ds["fosberg"] = calc_fosberg(ds["t2m"] - 273.15, ds["rh"], ds["wind"])
ds

<xarray.Dataset> Size: 16GB
Dimensions:     (valid_time: 648988, latitude: 26, longitude: 30)
Coordinates:
  * latitude    (latitude) float64 208B 38.78 38.53 38.28 ... 33.03 32.78 32.53
  * longitude   (longitude) float64 240B -124.4 -124.2 -123.9 ... -117.4 -117.2
  * valid_time  (valid_time) datetime64[ns] 5MB 1951-01-01 ... 2025-01-13T03:...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    d2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    rh          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    wind        (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    fosberg     (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-16T21:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
ds.to_zarr("era5_derived.zarr", mode="w")

/home/mitch/mamba/envs/pygeo/lib/python3.12/site-packages/zarr/api/asynchronous.py:197: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
